![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_RADIOLOGY.ipynb)

# `ner_radiology` **Models**

Pretrained named entity recognition deep learning model for radiology related texts and reports.

## 1. Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## 2. Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## 3. Select the model and construct the pipeline

**Create the pipeline**

In [ ]:
document_assembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained() \
      .setInputCols(["document"]) \
      .setOutputCol("sentence") 

tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")


word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("word_embeddings")

clinical_ner = medical.NerModel.pretrained("ner_radiology", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "word_embeddings"]) \
      .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setGreedyMode(True)

nlpPipeline = Pipeline(
    stages = [
        document_assembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,]) 

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_radiology download started this may take some time.
[OK!]


## 4. Create example inputs

In [ ]:
sample_text = [
"""Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma.""",

""" Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new.  The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine""",

# Acute fracture 
""" Clinical History: r/o acute process Report: LEFT WRIST AND LEFT FOREARM:  FINDINGS:  Three standard views of left wrist and two views of left forearm were obtained. The latter consists of lateral and oblique views without straight  AP view.  
Impression: 1. There is what appears to be Smith’s fracture affecting the left wrist. It  consists of impacted comminuted fracture of distal left radial metaphysis having dorsal angulation at the fracture site as well as avulsion fracture of  ulnar styloid process. The fracture affects neighboring radiocarpal as well  as inferior radioulnar articulations.  
2. Small cortical avulsion fracture is also noted on lateral view of wrist at carpo-metacarpal articulation, probably at 3rd carpometacarpal articulation. NOTE: Presence of an accessory ossicle dorsally opposing the triquetrum  or lunate. 
3. Low-grade osteoarthrosis at left 1st carpo-metacarpal articulation as  well as intermetacarpal articulation between 1st and 2nd.  
4. The rest of radius and ulna are normal.
Procedure: KNEE 3 VIEWS """,

# Fracture negation 
"""Procedure: KNEE 3 VIEWS Clinical History: S/p fall from ladder with right hip fracture. Since then also with right knee swelling and knee that “gives out”. Prior films showed  effusion and narrowed joint space. Appreciate re-evaluation. 
Report: RIGHT KNEE THREE VIEWS, Impression: There is no acute fracture or dislocation. Mild patchy osteopenia. Probable small volume suprapatellar joint effusion. Mild patellar spurring.  """,

# Negation of acute fraction by age of fracture
"""Procedure: CALCANEOUS 2 VIEWS, Clinical History: SP FX CALCANEOUS, Report: See impression. Impression: Compared to the prior study of 1/12/99 there is no significant change in  the deformity of the calcaneus consistent with an old fracture which has undergone complete healing. The adjacent bony structures appear intact.  """,

# Implicit negation 
""" Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please eval thanks. Report: See Impression. Impression: BILATERAL FEET: No traumatic, arthritic, or developmental abnormality  is discernible. """
]

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text,StringType()).toDF('text')

df.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measurin...|
| Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is ne...|
| Clinical History: r/o acute process Report: LEFT WRIST AND LEFT FOREARM:  FINDINGS:  Three stand...|
|Procedure: KNEE 3 VIEWS Clinical History: S/p fall from ladder with right hip fracture. Since the...|
|Procedure: CALCANEOUS 2 VIEWS, Clinical History: SP FX CALCANEOUS, Report: See impression. Impres...|
| Procedure: FOOT 3 OR MORE VIEWS, Clinical History: 49 yo b m w cc of bilateral foot pain. please...|
+------------------------------------------------------------------------

## 5. Use the pipeline to create outputs

In [ ]:
result = nlpPipeline.fit(df).transform(df)

In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata, )).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("entity")).show()

+--------------------+-----+---+--------------------+
|               chunk|begin|end|              entity|
+--------------------+-----+---+--------------------+
|    Bilateral breast|    0| 15|            BodyPart|
|          ultrasound|   17| 26|         ImagingTest|
|          ovoid mass|   78| 87|     ImagingFindings|
|     0.5 x 0.5 x 0.4|  113|127|        Measurements|
|                  cm|  129|130|               Units|
|anteromedial aspe...|  163|202|            BodyPart|
|                mass|  210|213|     ImagingFindings|
|isoechoic echotex...|  228|248|     ImagingFindings|
|              muscle|  266|271|            BodyPart|
| internal color flow|  294|312|     ImagingFindings|
|benign fibrous ti...|  334|354|     ImagingFindings|
|              lipoma|  361|366|Disease_Syndrome_...|
|        Cardiac size|    1| 12|     ImagingFindings|
|                left|   41| 44|           Direction|
|             pleural|   46| 52|            BodyPart|
|            effusion|   54|

## 6. Visualize results

In [ ]:
from sparknlp_display import NerVisualizer

ner_viz = NerVisualizer()


    
for j in range(len(sample_text)):
    ner_viz.display(result = result.collect()[j], label_col = "ner_chunk")
    print("\n\n")